# HexMagic

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

In [ ]:
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
sys.path.append(".")
sys.path.append("..")
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.primitives import Hex, MapCord 


In [ ]:
#| export

sampleHex = Hex(radius=50,center=MapCord(100,100))

sampleHex.vertices(), "\n-- We round edges so we can compare --\n", sampleHex.edges()

([(143.3,75.0),
  (143.3,125.0),
  (100.0,150.0),
  (56.7,125.0),
  (56.7,75.0),
  (100.0,50.0)],
 '\n-- We round edges so we can compare --\n',
 {(56, 75, 56, 125),
  (56, 75, 100, 50),
  (56, 125, 100, 150),
  (100, 50, 143, 75),
  (100, 150, 143, 125),
  (143, 75, 143, 125)})

In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
#some drawing setup 
hexStyle = StyleCSS("HexStyle",fill="yellow",stroke="blue",stroke_width=2)
canvas = SVGBuilder()
canvas.width=200 ;canvas.height=200
canvas.add_style(hexStyle)

#add our hex to the canvas
#some drawing setup 


#add our hex to the canvas
sampleHex = Hex(radius=50,center=MapCord(100,100),style=hexStyle)
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()

#| export
Hex, short for hexagon, is a six-sided polygon that can be laid out in a honeycomb pattern that is space-efficient and strong. It also has a nicer perimeter than a square. This library groups them in a hexgrid, a honeycomb type structure

In [ ]:
#| export
from HexMagic.primitives import HexGrid

def sampleGrid(numRings = 2, fill = "#ff7b00ff"):
    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)

    #Our new class
    grid = HexGrid.create_centered(numRings,radius=30,style=hexStyle)

    #make sure we are ready to draw
    grid.update()
    return grid

sampleGrid().builder.show()

#| export
The `HexGrid` has a list of `Hex`, and we initialize them with a common style. The hexes are ordered starting from the top left.

In [ ]:
#| export

def sampleGrid(hexDim = 2, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.create_centered(hexDim,radius=40,style=hexStyle)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        if makeLabels:
            grid.hexes[i].label = str(i) 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    return grid
    

grid = sampleGrid(makeLabels=True)
grid.builder.show()

#| export
### Absolute versus Realtive positions

This integer base system is great for absolute postioning, but is problematic from a realtive one. If we look at hex 12 two of its neighbors (11,13) are off by one. But the rest are going to be tricky since we need the number of rows and columns. Also even rows are different than odd ones.

What we need are directions.







In [ ]:
#| export
from HexMagic.primitives import HexPosition , MapPath
grid = sampleGrid()

#mark the treasure
grid.hexes[grid.middle()].label = "X"

#lets have a directions layer
arrowLayer = ""

#Use the realtive position class called HexPosition
for position in HexPosition.directions():
    i = grid.hexposition_to_index(position, 12)
    grid.hexes[i].label = position.label
    arrowLayer += grid.arrow(12,i)

grid.update()

#we can add a layer or update one in a builder using the adjust method
grid.builder.adjust("arrows",arrowLayer)

grid.builder.show()

In [ ]:
 eastDir = HexPosition.direction("E")
 eastDir,  2 * eastDir

(HexPosition(1, 0, -1, 'E'), HexPosition(2, 0, -2))

In [ ]:
#| export
def showEast(middleIndex=12):
    grid = sampleGrid()

    #lets have a directions layer
    arrowLayer = ""
    
    #Mark our starting point
    grid.hexes[middleIndex].label = str(middleIndex)

    #Get a Direction
    eastDir = HexPosition.direction("E")

    #go part way
    i = grid.hexposition_to_index(eastDir, middleIndex)
    grid.hexes[i].label = "near"
    arrowLayer += grid.arrow(middleIndex,i)

    #keep going
    farEast = 2 * eastDir 
    j = grid.hexposition_to_index(farEast, middleIndex)
    grid.hexes[j].label = "far"
    arrowLayer += grid.arrow(i,j)

    grid.update()
    grid.builder.adjust("arrows",arrowLayer)
    return grid.builder.show()
showEast(12)


#| export
The nice part is that there is nothing special about where we start. It doesn't have to be the center

In [ ]:
#| export
showEast(6)

#| export
A common journey that you might want to do is to go out a certain distance and walk around in a ring.

In [ ]:
@patch
def ring(self: HexPosition, radius: int, clockwise: bool = True) -> list['HexPosition']:
    """Generate all HexPositions in a ring at given radius from center."""
    if radius == 0:
        return [self]
    
    results = []
    current = self + radius * HexPosition.direction("E")
    
    # SW for counter-clockwise, SE for clockwise
    direction = HexPosition.direction("E").rotate(2 if not clockwise else -2)
    rotation = 1 if not clockwise else -1
    
    for side in range(6):
        for _ in range(radius):
            results.append(current)
            current = current + direction
        direction = direction.rotate(rotation)
    
    return results


In [ ]:
#| export
def showRing(middleIndex=12,ring=1):
    grid = sampleGrid()
    
    grid.hexes[middleIndex].label = "X"


    #lets have a directions layer
    arrowLayer = ""
    #Get a Direction
    eastDir = HexPosition.direction("E")

    #go out to the edge
    s = middleIndex
    for k in range(ring):
        i = grid.hexposition_to_index(eastDir, s)
        grid.hexes[i].label = f"r {k+1}"
        arrowLayer += grid.arrow(s,i)
        s = i

    origin = HexPosition.origin()

    #now lets go through the positions. by default it starts in the east like the sunrise.
    positions = origin.ring(ring)
    for pos in positions:
        i = grid.hexposition_to_index(pos,middleIndex ) 
        grid.hexes[i].label = f"r {pos.label}"
        if s != i:
            arrowLayer += grid.arrow(s,i)
        s = i

    grid.update()
    grid.builder.adjust("arrows",arrowLayer)
    return grid.builder.show()
    
showRing(12)

In [ ]:
#| export
showRing(12,ring=2)

#| export
### Coordinates

When representing hex directions, we need a coordinate system with several key properties:

1. **Additive**: Directions should combine naturally (e.g., two "east" moves = moving two hexes east)
2. **Reversible**: Every direction has a clear opposite (West is the reverse of East)
3. **Computationally efficient**: Operations should be fast and avoid floating-point errors

Since hexagons exist in the x,y plane, let's consider our options:

#### Cartesian Coordinates
The familiar x,y system works well for horizontal movements, but diagonal vectors involve factors of √3, requiring floating-point arithmetic and introducing potential rounding errors.

#### Polar Coordinates
Angles are clean multiples of 60°, but constant conversion between polar and Cartesian for rendering makes this impractical.

#### Dive a bit deeper
We have three pairs of directions (E/W,NW/SE,NE/SW) so we are going to need at 2 bits to repesent them. Lets build them

1. East can be (1, 0)
2. This makes West (-1, 0)
3. Let's make Southeast (0, 1)
4. Which makes Northwest (0, -1)
5. To get to Northeast using what we have built already. we go east (1,0), and then north west (0, -1) for (1,-1)
6. And that makes Southwest the opposite of Northeast (-1,1)




In [ ]:
def axisGrid(hexDim = 2, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.create_centered(hexDim,radius=40,style=hexStyle)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        pos = grid.index_to_hexposition(i)
        if makeLabels:
            grid.hexes[i].label = f"{pos.q}, {pos.r}" 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    baseDir = [HexPosition.direction(x) for x in ["E","W","NE","SW"]]
    outs = [HexPosition.direction(x) for x in ["E","W","NE","SW"]]
    for k in range(hexDim-1):
        for i in range(len(baseDir)):
            outs[i] += baseDir[i]

    arrowLayer = ""
    for pos in outs:
        index = grid.hexposition_to_index(pos)
        arrowLayer += grid.arrow(grid.middle(),index)

    grid.builder.adjust("axis",arrowLayer)
    return grid
    

grid = axisGrid(4,makeLabels=True)
grid.builder.show()

#| export
#### Cube Coordinates
What we have actually created are the first two digits of **cube coordinates** - a three-axis system (q, r, s) where:
- East: (1, 0, -1)
- West: (-1, 0, 1)
- Southeast: (0, 1, -1)
- Northwest: (0, -1, 1)
- Northeast: (1, -1, 0)
- Southwest: (-1, 1, 0)

The key constraint: **q + r + s = 0** always.

#### Why Cube Coordinates?

1. **Integer arithmetic only**: All operations use integers - no floating-point errors
2. **Natural addition**: Adding positions is simple component-wise addition
3. **Built-in validation**: The q + r + s = 0 constraint acts like double-entry accounting, catching errors immediately
4. **Distance calculation**: Manhattan distance in cube space equals hex distance: `(|q| + |r| + |s|) / 2`
5. **Rotation and reflection**: These operations become simple transformations of the three coordinates
6. **Line drawing**: Interpolating between hexes is straightforward with linear interpolation
7. **Range queries**: Finding all hexes within N steps is elegant: all positions where `(|q| + |r| + |s|) / 2 ≤ N`

The redundancy of the third coordinate isn't waste - it's a feature that makes many algorithms simpler and more robust.


**Flip** inverts all coordinates:
```
flip(q, r, s) = (-q, -r, -s)
```

**Geometric meaning**: Rotation by 180° around the origin, or equivalently, reflection through the origin.

**Example**:

In [ ]:
def flip(h):
    return (-HexPosition.direction(h)).desc()


flip("E")

'W'


**Geometric meaning**: Rotation by 180° around the origin, or equivalently, reflection through the origin.

**Example**:
```
flip(1, 0, -1) = (-1, 0, 1)
E → W (East becomes West)
```

**Key property**: Flip creates **opposite pairs**. Every direction has an opposite, and moving in direction D then flip(D) returns you to the start:
```
H + D + flip(D) = H + D - D = H
```

This is why addition and subtraction work naturally:
```
(2, 3, -5) - (1, 0, -1) = (2, 3, -5) + flip(1, 0, -1) 
                        = (2, 3, -5) + (-1, 0, 1)
                        = (1, 3, -4)
```

**Algebraic significance**: Flip gives us **additive inverses**, a requirement for forming a group (and later, a ring).

### 2. Roll (60° Rotation)

**Roll** cycles the coordinates:
```
roll(q, r, s) = (-s, -q, -r)
```

**Why the negations?** We need to preserve the constraint `q + r + s = 0`:
```
-s + (-q) + (-r) = -(s + q + r) = -0 = 0 ✓
```

In [ ]:

def roll(h):
    """ Geometric meaning**: Rotation by 60° counter-clockwise around the origin."""
    pos = -HexPosition.direction(h)
    return HexPosition(pos.s,pos.q,pos.r).desc()
   

#Rolling six times returns to the original position, giving us **cyclic order**.
start = "E"
for I in range(6):
    print(start)
    start = roll(start)

#**Algebraic significance**: Roll generates **rotational symmetry**, the six-fold symmetry that makes hexagons special.


E
NE
NW
W
SW
SE


Notice that `roll³(E) = flip(E)`, so roll alone could generate everything. But having **both** operations makes the structure clearer:
- **Roll** gives us rotation around the circle
- **Flip** gives us opposite pairs

### The Group Structure

The six directions form a **cyclic group** of order 6 under composition, isomorphic to **ℤ/6ℤ** (integers modulo 6).

But here's the magic: **positions** (not just directions) form something richer—a structure where:
1. We can **add** positions: `(q₁, r₁, s₁) + (q₂, r₂, s₂) = (q₁+q₂, r₁+r₂, s₁+s₂)`
2. We can **scale** by integers: `n · (q, r, s) = (nq, nr, ns)`
3. We have **additive inverses** (flip): `-(q, r, s) = (-q, -r, -s)`

This makes hex positions a **ℤ-module** (like a vector space, but over integers instead of real numbers).

## Why Three Coordinates? The Algebraic Answer

Here's the deep reason: **cube coordinates embed the 2D hexagonal lattice into 3D space in a way that preserves all symmetries**.

### Option 1: Two Coordinates (Axial)

We could use `(q, r)` and compute `s = -q - r`. This works, but:
- Addition is still natural: `(q₁, r₁) + (q₂, r₂) = (q₁+q₂, r₁+r₂)`
- But **rotation becomes asymmetric**:
  ```
  roll(q, r) = (q + r, -q)   [derived from roll(q, r, -q-r)]
  ```
  This formula isn't obviously a rotation! The symmetry is hidden.
Notice that `roll³(E) = flip(E)`, so roll alone could generate everything. But having **both** operations makes the structure clearer:
- **Roll** gives us rotation around the circle
- **Flip** gives us opposite pairs

### The Group Structure

The six directions form a **cyclic group** of order 6 under composition, isomorphic to **ℤ/6ℤ** (integers modulo 6).

But here's the magic: **positions** (not just directions) form something richer—a structure where:
1. We can **add** positions: `(q₁, r₁, s₁) + (q₂, r₂, s₂) = (q₁+q₂, r₁+r₂, s₁+s₂)`
2. We can **scale** by integers: `n · (q, r, s) = (nq, nr, ns)`
3. We have **additive inverses** (flip): `-(q, r, s) = (-q, -r, -s)`

This makes hex positions a **ℤ-module** (like a vector space, but over integers instead of real numbers).

## Why Three Coordinates? The Algebraic Answer

Here's the deep reason: **cube coordinates embed the 2D hexagonal lattice into 3D space in a way that preserves all symmetries**.

### Option 1: Two Coordinates (Axial)

We could use `(q, r)` and compute `s = -q - r`. This works, but:
- Addition is still natural: `(q₁, r₁) + (q₂, r₂) = (q₁+q₂, r₁+r₂)`
- But **rotation becomes asymmetric**:
  ```
  roll(q, r) = (q + r, -q)   [derived from roll(q, r, -q-r)]
  ```
  This formula isn't obviously a rotation! The symmetry is hidden.


### Option 2: Three Coordinates (Cube)

With `(q, r, s)`:
- Addition: `(q₁, r₁, s₁) + (q₂, r₂, s₂) = (q₁+q₂, r₁+r₂, s₁+s₂)` ✓
- Flip: `(-q, -r, -s)` (negate all) ✓  
- Roll: `(-s, -q, -r)` (permute and negate) ✓

All three operations treat the coordinates **symmetrically**. The three-fold symmetry of the constraint `q + r + s = 0` matches the six-fold symmetry of the hexagon (6 = 2 × 3).

### The Lie Algebra Connection

For those familiar with advanced mathematics: the constraint `q + r + s = 0` defines a **2D subspace** of ℤ³ that's **closed under rotation**. This is the **root lattice** of type A₂ (or equivalently, the weight lattice of SU(3)).

The roll operation is a **discrete rotation** preserving this lattice. Having all three coordinates lets us express this rotation as a simple permutation matrix, revealing the underlying symmetry group.

## Ring Structure: Combining Operations

Cube coordinates form an **abelian group** under addition:
- **Closure**: Sum of two hex coords is a hex coord (constraint preserved)
- **Associativity**: `(A + B) + C = A + (B + C)`
- **Identity**: `(0, 0, 0)` is the origin
- **Inverses**: flip gives `-A` for every `A`
- **Commutativity**: `A + B = B + A`

We can also define **scalar multiplication** by integers:
```
n · (q, r, s) = (nq, nr, ns)
```

This makes cube coordinates a **ℤ-module**, which is the algebraic structure that captures:
- Linear combinations of positions
- Scaling by integer factors
- All while preserving the hex lattice


## Algorithims

### 1. Distance Calculation

The **distance** between two hexes is:
```python
@patch
def distance(self: HexPosition, other: HexPosition) -> int:
    """Cube coordinate distance using numpy"""
    return int(np.abs(self._coords - other._coords).sum() // 2)
```

With three coordinates, this formula is **symmetric**. With axial `(q, r)`, you'd need:
```python
def distance_axial(h1, h2):
    dq = h1.q - h2.q
    dr = h1.r - h2.r
    return max(abs(dq), abs(dr), abs(dq + dr))
```
Less intuitive and asymmetric.

### 2. Rotation

Rotating a hex coordinate by 60° counter-clockwise:
```python
@patch
def rotate_left(self: HexPosition) -> 'HexPosition':
    """Rotate 60° counter-clockwise: (q,r,s) -> (-s,-q,-r)"""
    return HexPosition(*(-np.roll(self._coords, 1)))
```

Rotating by 60° clockwise:
```python
@patch
def rotate_right(self: HexPosition) -> 'HexPosition':
    """Rotate 60° clockwise: (q,r,s) -> (-r,-s,-q)"""
    return HexPosition(*(-np.roll(self._coords, -1)))
```

Both are **simple permutations**. With two coordinates, rotation formulas are more complex.


To draw a line between hexes, we use **linear interpolation** in cube coordinates:
```python
@patch
def line_to(self: HexPosition, other: HexPosition) -> list[HexPosition]:
    """Get line of hexes from self to other using linear interpolation"""
    n = self.distance(other)
    if n == 0:
        return [self]

    results = []
    for i in range(n + 1):
        t = i / n
        # Lerp in cube coordinates using numpy
        coords = self._coords + (other._coords - self._coords) * t
        results.append(HexPosition._cube_round(*coords))

    return results
```

In [ ]:

def hexes_in_range(n):
    """ finding all hexes within distance N from origin: """
    results = []
    for q in range(-n, n + 1):
        for r in range(max(-n, -q - n), min(n, -q + n) + 1):
            s = -q - r
            results.append(HexPosition(q, r, s))
    return results

hexes_in_range(3)

[HexPosition(-3, 0, 3),
 HexPosition(-3, 1, 2),
 HexPosition(-3, 2, 1),
 HexPosition(-3, 3, 0),
 HexPosition(-2, -1, 3),
 HexPosition(-2, 0, 2),
 HexPosition(-2, 1, 1),
 HexPosition(-2, 2, 0),
 HexPosition(-2, 3, -1),
 HexPosition(-1, -2, 3),
 HexPosition(-1, -1, 2),
 HexPosition(-1, 0, 1),
 HexPosition(-1, 1, 0),
 HexPosition(-1, 2, -1),
 HexPosition(-1, 3, -2),
 HexPosition(0, -3, 3),
 HexPosition(0, -2, 2),
 HexPosition(0, -1, 1),
 HexPosition(0, 0, 0),
 HexPosition(0, 1, -1),
 HexPosition(0, 2, -2),
 HexPosition(0, 3, -3),
 HexPosition(1, -3, 2),
 HexPosition(1, -2, 1),
 HexPosition(1, -1, 0),
 HexPosition(1, 0, -1),
 HexPosition(1, 1, -2),
 HexPosition(1, 2, -3),
 HexPosition(2, -3, 1),
 HexPosition(2, -2, 0),
 HexPosition(2, -1, -1),
 HexPosition(2, 0, -2),
 HexPosition(2, 1, -3),
 HexPosition(3, -3, 0),
 HexPosition(3, -2, -1),
 HexPosition(3, -1, -2),
 HexPosition(3, 0, -3)]

#| export
### Disscussion

#### Advanced Topic: Why Roll Uses Negation

A common question: why is roll `(-s, -q, -r)` instead of `(s, q, r)`?

**Answer**: We need `q + r + s = 0` to remain true after rolling.

If we tried `roll'(q, r, s) = (s, q, r)`:
```
s + q + r = -(q + r + s) = -0 = 0  (wait, that works!)
```

Actually, `(s, q, r)` does preserve the constraint! But:
```
roll'(1, 0, -1) = (-1, 1, 0)
```

This is **clockwise** rotation, not counter-clockwise. The conventional choice is:
- **Counter-clockwise**: `(-s, -q, -r)`
- **Clockwise**: `(-r, -s, -q)`

Both preserve the constraint. We choose counter-clockwise to match standard mathematical convention (positive angles rotate counter-clockwise).

The negations ensure the **direction** of rotation aligns with our coordinate system.

#### Comparison to Modular Arithmetic

You mentioned the **modular operator** as a more common way to create cyclic structures. Let's compare:

##### Modular Arithmetic (ℤ/6ℤ)

In **ℤ mod 6**, we have six elements: {0, 1, 2, 3, 4, 5}

Operations:
- **Addition**: `(a + b) mod 6`
- **Negation**: `(-a) mod 6 = (6 - a) mod 6`
- **Multiplication**: `(a · b) mod 6`

Example: Directions as integers
```
E = 0, NE = 1, NW = 2, W = 3, SW = 4, SE = 5
```

To rotate by 60°: `new_dir = (old_dir + 1) mod 6`  
To flip (180°): `new_dir = (old_dir + 3) mod 6`

**Advantages**:
- Single number to store per direction
- Fast operations with bit masking (`& 0x3F` for mod 64)

**Disadvantages**:
- **Positions** can't be represented this way! `(0 + 2) mod 6 = 2`, but the position "2 east" is different from "1 east + 1 northeast"
- No natural **distance metric**
- No **interpolation** or continuous paths

##### Cube Coordinates

Positions: `(q, r, s)` with `q + r + s = 0`

Operations:
- **Addition**: Component-wise
- **Negation**: Component-wise
- **Rotation**: Permutation with sign

**Advantages**:
- Positions AND directions in same framework
- Natural distance, interpolation, range queries
- Geometric intuition (points in 3D space)

**Disadvantages**:
- 3× memory vs single direction index
- Requires maintaining constraint


In [ ]:
def axisGrid(hexDim = 2, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.create_centered(hexDim,radius=40,style=hexStyle)
    

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        pos = grid.index_to_hexposition(i)
        if makeLabels:
            grid.hexes[i].label = f"{pos.q}, {pos.r}, {pos.s}" 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    

    startPos = HexPosition(4,-4,0)
    endPos = HexPosition(4,0,-4)
    baseDir = [HexPosition.direction(x) for x in ["E","W","NE","SW"]]
    outs = [HexPosition.direction(x) for x in ["E","W","NE","SW"]]
    for k in range(hexDim-1):
        for i in range(len(baseDir)):
            outs[i] += baseDir[i]

    arrowLayer = ""
    
    arrowLayer += grid.arrow(grid.hexposition_to_index(startPos),grid.hexposition_to_index(endPos))

    grid.builder.adjust("axis",arrowLayer)
    return grid
    

grid = axisGrid(4,makeLabels=True)
grid.builder.show()

In [ ]:
#| export
#??HexPosition.line_to

In [ ]:
#| export
def line_demo(start_idx=20, end_idx=8):
    """Demo showing line drawing between two hexes using cube coordinates"""
    grid = sampleGrid()
    # Clear the axis arrows from sampleGrid
    grid.builder.adjust("axis", "")
    
    # Mark start and end
    grid.hexes[start_idx].label = "Start"
    grid.hexes[end_idx].label = "End"
    
    # Get the hex positions. This defaults to postion 0
    start_pos = grid.index_to_hexposition(start_idx)
    end_pos = grid.index_to_hexposition(end_idx)
    
    # Draw a line between them using cube coordinate interpolation
    path_positions = start_pos.line_to(end_pos)
    
    arrowLayer = ""
    prev_idx = start_idx
    
    for i, pos in enumerate(path_positions[1:], 1):  # Skip first (it's the start)
        curr_idx = grid.hexposition_to_index(pos)
        
        # Label intermediate hexes
        if curr_idx != end_idx and curr_idx >= 0:
            grid.hexes[curr_idx].label = str(i)
        
        # Draw arrow if valid index
        if curr_idx >= 0:
            arrowLayer += grid.arrow(prev_idx, curr_idx)
            prev_idx = curr_idx
    
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)
    return grid.builder.show()

line_demo(20, 8)


In [ ]:
#| export
def range_demo( max_distance=2):
    """Demo showing all hexes within a certain distance"""
    
    grid = sampleGrid(3, fill="lightgray")
    center_idx = grid.middle()

    # Mark center
    grid.hexes[center_idx].label = "0"
    
    # Get center position
    center_pos = grid.index_to_hexposition(center_idx)
    
    # Color hexes by distance
    colors = [StyleCSS(x,fill=x,stroke="blue",stroke_width=2) for x in ["green","yellow", "orange", "red", "purple"]]
    for color in colors:
        grid.builder.add_style(color)
    
    for distance in range(1, max_distance + 1):
        # Get all positions at this distance
        positions = center_pos.ring(distance)
        
        for pos in positions:
            try:
                idx = grid.hexposition_to_index(pos, center_idx)
                if 0 <= idx < len(grid.hexes):
                    grid.hexes[idx].label = str(distance)
                    grid.hexes[idx].style = colors[distance-1]
                    # You could change fill color here if we modify the hex style
            except:
                pass  # Position outside grid
    
    grid.update()
    return grid.builder.show()

range_demo(3)


In [ ]:
#| export
def rotation_demo(center_idx=12):
    """Demo showing rotation around a center hex"""
    grid = sampleGrid()
    # Clear the axis arrows from sampleGrid
    grid.builder.adjust("axis", "")
    
    grid.hexes[center_idx].label = "Center"
    
    # Start with a position to the east
    start_pos = HexPosition.direction("E")
    start_idx = grid.hexposition_to_index(2 * start_pos , center_idx)
    grid.hexes[start_idx].label = "0°"
    
    arrowLayer = ""
    prev_idx = start_idx
    
    # Rotate around 60° at a time
    for i in range(1, 6):
        # Rotate the position
        rotated_pos = 2 * start_pos.rotate(-i)
        curr_idx = grid.hexposition_to_index(rotated_pos, center_idx)
        
        grid.hexes[curr_idx].label = f"{i*60}°"
        arrowLayer += grid.arrow(prev_idx, curr_idx)
        prev_idx = curr_idx
    
    # Close the loop
    arrowLayer += grid.arrow(prev_idx, start_idx)
    
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)
    return grid.builder.show()

rotation_demo(12)


In [ ]:
def demo_region_spiral_with_arrows():
    """Demo showing a region growing in a spiral pattern with arrows"""
    grid = sampleGrid(4, fill="lightgray")
    
    # Create gradient of colors
    colors = StyleCSS.seaborn("BuGn",37)
    
    arrow_style = StyleCSS("spiral_arrow", stroke="#333333", stroke_width=2, fill="none")
    
    for color in colors:
        grid.builder.add_style(color)
    grid.builder.add_style(arrow_style)
    
    center = grid.middle()
    
    # Use the spiral method from HexPosition
    spiral_positions = HexPosition.origin().spiral(3)
    
    arrow_layer = ""
    prev_idx = None
    
    for i, pos in enumerate(spiral_positions):
        idx = grid.hexposition_to_index(pos, center)
        if 0 <= idx < len(grid.hexes):
            grid.hexes[idx].style = colors[i % len(colors)]
            grid.hexes[idx].label = str(i)
            
            # Draw arrow from previous hex to current
            if prev_idx is not None and prev_idx >= 0:
                arrow_layer += grid.arrow(prev_idx, idx, style=arrow_style)
            
            prev_idx = idx
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    return grid.builder.show()

demo_region_spiral_with_arrows()

#| export
# Regions

We can combine both ideas of using integer indexes to access data and this ring system to check neighbors. A group of hex indexes can be combined into a set called hexregion.
Coming soon to you

In [ ]:
#| export
from HexMagic.primitives import HexRegion

In [ ]:
grid = sampleGrid(3, fill="lightgray")
perimeter_style=StyleCSS("perimeter_path", fill="red",  stroke="#ba3ca3ff", stroke_width=3)

region = HexRegion(set([grid.middle(),grid.middle()+1]), grid) 

paths = region.trace_perimeter(style=perimeter_style)
grid.builder.add_style(perimeter_style)
overlay = "" + "".join([x.drawPloy() for x in paths])
grid.builder.adjust("regionBorder",overlay)
grid.builder.show()

In [ ]:
grid = sampleGrid(3, fill="lightgray")
perimeter_style=StyleCSS("perimeter_path", fill="red",  stroke="#ba3ca3ff", stroke_width=3)

region = HexRegion(set([grid.middle(),grid.middle()+1,int(grid.middle()/2)]), grid) 

paths = region.trace_perimeter(style=perimeter_style)
grid.builder.add_style(perimeter_style)
overlay = "" + "".join([x.drawPloy() for x in paths])
grid.builder.adjust("regionBorder",overlay)
grid.builder.show()

We can use regions to decorate new layers for our canvas. 

In [ ]:
def demo_region_erosion():
    """Demo showing a region eroding from outside in"""
    grid = sampleGrid(4, fill="lightgray")

    # We can do this first because we are going to add layers on top of it
    grid.update()

    stages = [
        StyleCSS(f"erode{i}", fill=color, stroke="blue", stroke_width=2)
        for i, color in enumerate(["#2c3e50", "#34495e", "#7f8c8d", "#95a5a6", "#bdc3c7"])
    ]


    
    for style in stages:
        grid.builder.add_style(style)
    
    center = grid.middle()
    large_region = HexRegion(
        set(grid.hexposition_to_index(pos, center) 
            for pos in HexPosition.origin().spiral(3)
            if grid.hexposition_to_index(pos, center) >= 0),
        grid
    )
    
    # First pass: create layers and count them
    current_region = large_region
    stage = 0

    while len(current_region.hexes) > 0 and stage < len(stages):
        boundaries = current_region.trace_perimeter(style=stages[stage])
        layerId = f"stage{stage}"
        grid.builder.adjust(layerId, "\n".join([x.drawPloy() for x in boundaries]))
        current_region = current_region.inside()
        stage += 1
   
    return grid.builder.show()

demo_region_erosion()

And then use the animation properties of layers to show growth.

In [ ]:
def demo_region_erosion():
    """Demo showing a region eroding from outside in"""
    grid = sampleGrid(4, fill="lightgray")
    grid.update()

    stages = [
        StyleCSS(f"erode{i}", fill=color, stroke="blue", stroke_width=2)
        for i, color in enumerate(["#2c3e50", "#34495e", "#7f8c8d", "#95a5a6", "#bdc3c7"])
    ]
    
    for style in stages:
        grid.builder.add_style(style)
    
    center = grid.middle()
    large_region = HexRegion(
        set(grid.hexposition_to_index(pos, center) 
            for pos in HexPosition.origin().spiral(3)
            if grid.hexposition_to_index(pos, center) >= 0),
        grid
    )
    
    # Timing configuration
    fade_in, stagger, hold, fade_out = 0.6, 0.4, 1.0, 0.5
    
    # Build up rings from outside in using set subtraction
    rings = []
    outer = large_region
    for i, style in enumerate(stages):
        inner = outer.inside()
        ring = outer - inner  # The ring at this level!
        if len(ring) == 0:
            break
        rings.append((f"stage{i}", ring, style, i))
        outer = inner
    
    # If there's a center left, add it as the last ring
    if len(outer) > 0 and len(rings) < len(stages):
        rings.append((f"stage{len(rings)}", outer, stages[len(rings)], len(rings)))
    
    # Draw and animate each ring
    for layerId, ring, style, stage_idx in rings:
        boundaries = ring.trace_perimeter(style=style)
        grid.builder.adjust(layerId, "\n".join([x.drawPloy() for x in boundaries]))
        
        is_last = (stage_idx == len(rings) - 1)
        grid.builder.animate_layer_full_cycle(
            layerId,
            fade_in_duration=fade_in,
            hold_duration=hold + (len(rings) - 1 - stage_idx) * stagger,
            fade_out_duration=fade_out if not is_last else 0,
            delay=stage_idx * stagger,
            easing="ease-out",
            loop=True
        )
    
    return grid.builder.show()

demo_region_erosion()


The animation system needs to be reworked so this is more of a future direction.

Animation is always a matter of taste. And sometimes the better visualization is not to use it.

In [ ]:
def demo_voronoi_growth_simple():
    """Simpler Voronoi growth using HexRegion list"""
    grid = sampleGrid(5, fill="lightgray")
    
    colors = [
        StyleCSS("region0", fill="#ff6b6b", stroke="darkred", stroke_width=0),
        StyleCSS("region1", fill="#4ecdc4", stroke="darkblue", stroke_width=0),
        StyleCSS("region2", fill="#95e1d3", stroke="darkgreen", stroke_width=0),
        StyleCSS("region3", fill="#f38181", stroke="darkorange", stroke_width=0),
    ]
    
    perimeter_style = StyleCSS("perimeter_path", fill="none", stroke="#333", stroke_width=3)
    arrow_style = StyleCSS("growth_arrow", stroke="#333333", stroke_width=2, fill="none")
    boundary_layer = ""
    for style in colors + [perimeter_style, arrow_style]:
        grid.builder.add_style(style)
    
    seeds = [
        grid.nCols * 1 + 1,
        grid.nCols * 1 + (grid.nCols - 2),
        grid.nCols * (grid.nRows - 2) + 1,
        grid.nCols * (grid.nRows - 2) + (grid.nCols - 2),
    ]
    
    regions = [HexRegion({seed}, grid) for seed in seeds]
    
    for i, seed in enumerate(seeds):
        grid.hexes[seed].style = colors[i]
        grid.hexes[seed].label = str(i)
    
    claimed = set(seeds)
    total = grid.nRows * grid.nCols
    arrow_layer = ""
    
    # Grow until everything is claimed
    for round_num in range(30):
        if len(claimed) >= total:
            break
        
        # Collect ALL unclaimed candidates and which regions want them
        candidates = {}  # hex_idx -> [(region_idx, source, dist), ...]
        
        for i, region in enumerate(regions):
            centroid_idx = region.centroid_hex()
            centroid_pos = grid.index_to_hexposition(centroid_idx)
            
            outside = region.outside()
            for candidate in outside.hexes:
                if candidate in claimed:
                    continue
                    
                neighbors = grid.neighborsOf(candidate)
                source = next((n for n in neighbors if n in region.hexes), None)
                
                if source is not None:
                    candidate_pos = grid.index_to_hexposition(candidate)
                    dist = candidate_pos.distance(centroid_pos)
                    
                    if candidate not in candidates:
                        candidates[candidate] = []
                    candidates[candidate].append((i, source, dist))
        
        if not candidates:
            break
        
        # Assign each candidate to closest region (or first if tied)
        for candidate, claimants in candidates.items():
            claimants.sort(key=lambda x: (x[2], x[0]))  # Sort by dist, then region idx for ties
            region_idx, source, dist = claimants[0]
            
            claimed.add(candidate)
            regions[region_idx] = HexRegion(regions[region_idx].hexes | {candidate}, grid)
            grid.hexes[candidate].style = colors[region_idx]
            arrow_layer += grid.arrow(source, candidate, style=arrow_style)
    
        for region in regions:
            paths = region.trace_perimeter(style=perimeter_style)
            boundary_layer += "".join([x.make_windy(iterations=2, offset_factor=0.2).svg() for x in paths])
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    grid.builder.adjust("boundaries", boundary_layer)
    return grid.builder.show()

demo_voronoi_growth_simple()


Any advice about HexRegions you would want to give as you introduce the concept to a new programmer? any other demos?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question! Here's advice I'd give when introducing `HexRegion` to a new programmer:

**Core Concepts to Emphasize**

1. **A Region is Just a Set of Indices** - The simplest mental model: `HexRegion` wraps a Python `set` of hex indices. All the power comes from what you can *do* with that set.

2. **Morphological Operations** - These are the real gems:
   - `outside()` - expand the region by one ring (dilation)
   - `inside()` - shrink by removing boundary hexes (erosion)
   - These compose! `region.outside().outside()` expands by 2

3. **Set Operations Work** - Since it's backed by a set, you can combine regions:
   ```python
   union = HexRegion(region_a.hexes | region_b.hexes, grid)
   intersection = HexRegion(region_a.hexes & region_b.hexes, grid)
   difference = HexRegion(region_a.hexes - region_b.hexes, grid)
   ```

4. **Perimeter Tracing is Non-Trivial** - The `trace_perimeter()` method handles disconnected regions (returns multiple paths) and properly closes boundaries.

**Additional Demo Ideas**

```python
def demo_flood_fill():
    """Flood fill from a seed, stopping at 'walls'"""
    grid = sampleGrid(4, fill="lightgray")
    
    # Create some "walls" (blocked hexes)
    walls = {20, 21, 22, 30, 31, 40, 41}
    wall_style = StyleCSS("wall", fill="#333", stroke="black", stroke_width=2)
    grid.builder.add_style(wall_style)
    for w in walls:
        grid.hexes[w].style = wall_style
    
    # Flood fill from center, avoiding walls
    seed = grid.middle()
    filled = {seed}
    frontier = {seed}
    
    fill_style = StyleCSS("filled", fill="#88ccff", stroke="blue", stroke_width=2)
    grid.builder.add_style(fill_style)
    
    while frontier:
        new_frontier = set()
        for idx in frontier:
            for neighbor in grid.neighborsOf(idx):
                if neighbor not in filled and neighbor not in walls:
                    filled.add(neighbor)
                    new_frontier.add(neighbor)
        frontier = new_frontier
    
    for idx in filled:
        grid.hexes[idx].style = fill_style
    
    grid.update()
    return grid.builder.show()
```

```python
def demo_region_subtract():
    """Show donut shape via set subtraction"""
    grid = sampleGrid(4, fill="lightgray")
    center = grid.middle()
    
    # Large disk minus small disk = ring
    large = set(grid.hexposition_to_index(pos, center) 
                for pos in HexPosition.origin().spiral(3)
                if grid.hexposition_to_index(pos, center) >= 0)
    small = set(grid.hexposition_to_index(pos, center) 
                for pos in HexPosition.origin().spiral(1)
                if grid.hexposition_to_index(pos, center) >= 0)
    
    donut = HexRegion(large - small, grid)
    
    donut_style = StyleCSS("donut", fill="#ff9966", stroke="darkorange", stroke_width=2)
    grid.builder.add_style(donut_style)
    
    for idx in donut.hexes:
        grid.hexes[idx].style = donut_style
    
    grid.update()
    paths = donut.trace_perimeter()
    # Note: donut has TWO perimeters - inner and outer!
    print(f"Number of boundaries: {len(paths)}")
    return grid.builder.show()
```

**Key Insight to Share**: The donut demo reveals something important - `trace_perimeter()` returns a *list* of paths because regions can have holes or be disconnected. This is why it's more powerful than just walking vertices.

Would you like me to run any of these demos?